### Training per-digit RF verifiers

In [2]:
import os
import numpy as np
import torch
import joblib
import foolbox as fb
from foolbox.attacks.base import MinimizationAttack
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# --- Setup ---
os.makedirs("Models and Data splits", exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

# --- Load data & model ---
X_train, X_test, y_train, y_test = joblib.load(
    "Models and Data splits/data_[SCALED] Train_Test_Splits.pkl"
)
model = torch.jit.load("Models and Data splits/lenet.pt").to(device).eval()
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

# --- Attack parameters ---
eps_inf = 0.1        
eps_l2 = 1.0        
pgd_steps = 100
deepfool_steps = 1000
cw_steps = 1000      
max_train_per_digit = 2000

eps_grid_linf = np.array([0.1, 0.2, 0.3], dtype=np.float32)
eps_grid_l2   = np.array([0.5, 1.0, 2.0, 3.0], dtype=np.float32)

# --- Define attacks ---
attacks_eps = [
    ("LinfPGD", fb.attacks.LinfPGD(steps=pgd_steps)),
    ("L2PGD",   fb.attacks.L2PGD(steps=pgd_steps)),
]

attacks_minim = [
    ("LinfDeepFool", fb.attacks.LinfDeepFoolAttack(steps=deepfool_steps)),
    ("L2DeepFool",   fb.attacks.L2DeepFoolAttack(steps=deepfool_steps)),
    ("CW-L2",        fb.attacks.L2CarliniWagnerAttack(steps=cw_steps)),
]

# --- Utilities ---

def _to_numpy(x):
    """Convert torch / eagerpy / list-like to np.ndarray."""
    if isinstance(x, np.ndarray):
        return x
    # torch tensor
    if "torch" in str(type(x)):
        return x.detach().cpu().numpy()
    # eagerpy or similar
    if hasattr(x, "numpy"):
        return x.numpy()
    # list or other sequence
    return np.array(x)

def _collect_from_pairs(pairs):
    """pairs: iterable of (adv, success). Returns (M, 784) or None."""
    collected = []
    for adv_eps, suc_eps in pairs:
        adv_np = _to_numpy(adv_eps)       # (..., C, H, W)
        suc_np = _to_numpy(suc_eps).astype(bool)

        if adv_np.size == 0:
            continue

        # Ensure success mask is (N,)
        if suc_np.ndim > 1:
            suc_np = suc_np.reshape(suc_np.shape[0], -1).any(axis=1)

        if adv_np.ndim == 5:
            # (E, N, C, H, W) -> loop outside, so shouldn't happen here
            raise RuntimeError("Unexpected 5D adv inside _collect_from_pairs")
        if adv_np.ndim != 4:
            continue

        if suc_np.shape[0] != adv_np.shape[0]:
            # shape mismatch, skip this eps-level
            continue

        if suc_np.any():
            sel = adv_np[suc_np]
            collected.append(sel.reshape(sel.shape[0], -1))

    if not collected:
        return None
    return np.vstack(collected)

def collect_from_eps_attacks(attack, name, fmodel, images, labels, eps):
    """
    For PGD-style attacks. Handles both array and list outputs.
    """
    try:
        raw, _, success = attack(fmodel, images, labels, epsilons=eps)

        # Case 1: list of per-eps arrays
        if isinstance(raw, list):
            pairs = zip(raw, success)
            return _collect_from_pairs(pairs)

        # Case 2: single array; maybe with eps-dim
        raw_np = _to_numpy(raw)
        suc_np = _to_numpy(success).astype(bool)

        if raw_np.ndim == 5:
            # (E, N, C, H, W)
            pairs = [(raw_np[ei], suc_np[ei]) for ei in range(raw_np.shape[0])]
            return _collect_from_pairs(pairs)
        elif raw_np.ndim == 4:
            # (N, C, H, W)
            return _collect_from_pairs([(raw_np, suc_np)])
        else:
            return None

    except Exception as e:
        print(f"  [!] {name} failed: {e}")
        return None

def collect_from_minim_attack(attack, name, fmodel, images, labels):
    """
    For MinimizationAttack subclasses (DeepFool, C&W).
    In your Foolbox, these require epsilons and may return lists.
    """
    try:
        if "Linf" in name:
            eps_grid = eps_grid_linf
        else:
            eps_grid = eps_grid_l2

        raw, _, success = attack(fmodel, images, labels, epsilons=eps_grid)

        # Case 1: list-of-arrays
        if isinstance(raw, list):
            pairs = zip(raw, success)
            return _collect_from_pairs(pairs)

        # Case 2: array with eps-dim or not
        raw_np = _to_numpy(raw)
        suc_np = _to_numpy(success).astype(bool)

        if raw_np.ndim == 5:
            pairs = [(raw_np[ei], suc_np[ei]) for ei in range(raw_np.shape[0])]
            return _collect_from_pairs(pairs)
        elif raw_np.ndim == 4:
            return _collect_from_pairs([(raw_np, suc_np)])
        else:
            return None

    except Exception as e:
        print(f"  [!] {name} (MinimizationAttack) failed: {e}")
        return None

# --- Training loop ---

results = {}
available_digits = []

print("Training 1-NN verifiers (per digit) on strong multi-attack adversarials...")

for digit in range(10):
    print(f"\n→ Digit {digit}")

    # Clean data for this digit
    X_real_full = X_train[y_train == digit]
    if len(X_real_full) == 0:
        print("  ⚠️ No training samples, skipping.")
        continue

    if max_train_per_digit is not None and len(X_real_full) > max_train_per_digit:
        idx = np.random.choice(len(X_real_full), max_train_per_digit, replace=False)
        X_real = X_real_full[idx]
    else:
        X_real = X_real_full

    y_real = np.ones(len(X_real))

    images = torch.tensor(X_real, dtype=torch.float32).view(-1, 1, 28, 28).to(device)
    labels = torch.full((len(images),), digit, dtype=torch.long, device=device)

    adv_list = []

    # PGD-based attacks
    print("  PGD-style attacks:")
    for name, atk in attacks_eps:
        print(f"    {name}...")
        use_eps = eps_l2 if "L2" in name else eps_inf
        X_adv = collect_from_eps_attacks(atk, name, fmodel, images, labels, use_eps)
        if X_adv is not None:
            print(f"      collected {X_adv.shape[0]} adv")
            adv_list.append(X_adv)
        else:
            print("      no successes")

    # Minimization attacks on subset
    sub_n = min(500, len(images))
    if sub_n > 0:
        idx_sub = np.random.choice(len(images), sub_n, replace=False)
        images_sub = images[idx_sub]
        labels_sub = labels[idx_sub]

        print("  Minimization attacks on subset:")
        for name, atk in attacks_minim:
            print(f"    {name}...")
            X_adv = collect_from_minim_attack(atk, name, fmodel, images_sub, labels_sub)
            if X_adv is not None:
                print(f"      collected {X_adv.shape[0]} adv")
                adv_list.append(X_adv)
            else:
                print("      no successes")

    if not adv_list:
        print("  ⚠️ No successful adversarials, skipping this digit.")
        continue

    X_adv_all = np.vstack(adv_list)
    y_adv_all = np.zeros(X_adv_all.shape[0])

    # Balance clean vs adv
    n_adv = X_adv_all.shape[0]
    n_clean = min(2 * n_adv, X_real.shape[0])
    if n_clean == 0:
        print("  ⚠️ No clean after balancing, skipping.")
        continue

    X_clean_sub = X_real[:n_clean]
    y_clean_sub = y_real[:n_clean]

    X_train_i = np.vstack([X_clean_sub, X_adv_all])
    y_train_i = np.concatenate([y_clean_sub, y_adv_all])

    #RF
    rf=RandomForestClassifier(n_estimators=49)
    rf.fit(X_train_i, y_train_i)
    

    out_path = f"Models and Data splits/rf_digit_{digit}_pixels.joblib"
    joblib.dump(rf, out_path)
    available_digits.append(digit)

    # --- quick sanity check with strong PGD attacks ---
    X_test_real = X_test[y_test == digit]
    if len(X_test_real) == 0:
        print("  ⚠️ No test samples.")
        continue

    max_test = min(1000, len(X_test_real))
    X_test_real = X_test_real[:max_test]

    t_images = torch.tensor(X_test_real, dtype=torch.float32).view(-1, 1, 28, 28).to(device)
    t_labels = torch.full((len(t_images),), digit, dtype=torch.long, device=device)

    X_test_adv_list = []
    for name, atk in attacks_eps:
        use_eps = eps_l2 if "L2" in name else eps_inf
        X_adv_t = collect_from_eps_attacks(atk, name, fmodel, t_images, t_labels, use_eps)
        if X_adv_t is not None:
            X_test_adv_list.append(X_adv_t)

    if not X_test_adv_list:
        print("  ⚠️ No test adversarials; skipping metrics.")
        continue

    X_adv_t = np.vstack(X_test_adv_list)
    n_t = min(len(X_test_real), X_adv_t.shape[0])

    X_eval = np.vstack([X_test_real[:n_t], X_adv_t[:n_t]])
    y_eval = np.concatenate([np.ones(n_t), np.zeros(n_t)])

    y_pred = rf.predict(X_eval)

    cm = confusion_matrix(y_eval, y_pred)
    acc = accuracy_score(y_eval, y_pred)
    prec = precision_score(y_eval, y_pred)
    rec = recall_score(y_eval, y_pred)
    f1 = f1_score(y_eval, y_pred)

    results[digit] = dict(
        confusion_matrix=cm,
        accuracy=acc,
        precision=prec,
        recall=rec,
        f1_score=f1,
    )

    print(f"  Digit {digit} verifier: Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}")

print("\n✅ Trained rf verifiers for digits:", available_digits)


Training 1-NN verifiers (per digit) on strong multi-attack adversarials...

→ Digit 0
  PGD-style attacks:
    LinfPGD...
      collected 611 adv
    L2PGD...
      collected 251 adv
  Minimization attacks on subset:
    LinfDeepFool...
      collected 967 adv
    L2DeepFool...
      collected 770 adv
    CW-L2...
      collected 893 adv
  Digit 0 verifier: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000

→ Digit 1
  PGD-style attacks:
    LinfPGD...
      collected 2000 adv
    L2PGD...
      collected 1849 adv
  Minimization attacks on subset:
    LinfDeepFool...
      collected 1500 adv
    L2DeepFool...
      collected 1420 adv
    CW-L2...
      collected 1389 adv
  Digit 1 verifier: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000

→ Digit 2
  PGD-style attacks:
    LinfPGD...
      collected 1094 adv
    L2PGD...
      collected 449 adv
  Minimization attacks on subset:
    LinfDeepFool...
      collected 1141 adv
    L2DeepFool...
      collected 938 adv
    CW-L2...
      collect

### Verification of adversarials from multiple attacks

In [4]:
import os
import torch
import numpy as np
import joblib
import foolbox as fb

# --- Setup ---
os.environ['GIT_PYTHON_GIT_EXECUTABLE'] = r'C:\Program Files\Git\bin\git.exe'
torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Load LeNet ---
print("Loading LeNet model...")
model = torch.jit.load("Models and Data splits/lenet.pt").to(device).eval()
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

# --- Load rf verifiers ---
print("Loading per-digit rf verifiers...")
rfs = {}
for digit in range(10):
    path = f"Models and Data splits/rf_digit_{digit}_pixels.joblib"
    if os.path.exists(path):
        rfs[digit] = joblib.load(path)
        print(f"  Loaded rf for digit {digit}")
    else:
        print(f"  ⚠️ No verifier for digit {digit}, will skip those predictions.")

if not rfs:
    raise RuntimeError("No rf verifiers found.")

# --- Load test samples for attack ---
print("Loading test samples for attack...")
saved_data = torch.load("Models and Data splits/selected_samples_for_attack.pt")
images = saved_data["images"].to(device)        # shape (N, 1, 28, 28), scaled [0,1]
labels = saved_data["labels"].to(device)

# --- Attack configuration ---
epsLinf = 0.1
epsL2 = 1.0
confidence_threshold = 0.90

attacks = {
    "FGSM (L∞)": fb.attacks.FGSM(),
    "BIM (L∞)": fb.attacks.LinfBasicIterativeAttack(steps=50),
    "BIM (L2)": fb.attacks.L2BasicIterativeAttack(steps=50),
    "PGD (L∞)": fb.attacks.LinfPGD(steps=100),
    "PGD (L2)": fb.attacks.L2PGD(steps=100),
    "C&W (L2)": fb.attacks.L2CarliniWagnerAttack(steps=1000),
    "DeepFool (L2)": fb.attacks.L2DeepFoolAttack(steps=500),
}

def verify_with_rf(image, predicted_digit):
    """
    Use the corresponding per-digit rf to decide if this sample is 'clean-looking'.
    Returns True if accepted, False if rejected.
    """
    if predicted_digit not in rfs:
        return False  # conservative: reject if no verifier available

    rf = rfs[predicted_digit]
    image_flat = image.detach().cpu().numpy().reshape(1, -1)
    proba_clean = rf.predict_proba(image_flat)[0, 1]  # class 1 = clean
    return proba_clean >= confidence_threshold

# --- Evaluation ---
results = {name: {"total": 0, "accepted": 0} for name in attacks}

print("\nTesting attacks with rf verification...")

for name, attack in attacks.items():
    print(f"\n→ {name}")

    eps = epsL2 if "L2" in name else epsLinf

    # Generate adversarial examples
    adv_raw, _, _ = attack(fmodel, images, labels, epsilons=eps)

    # Handle possible shape (eps, N, C, H, W)
    if adv_raw.ndim == 5:
        adv_images = adv_raw[0]
    else:
        adv_images = adv_raw

    adv_images = adv_images.to(device)
    N = adv_images.shape[0]

    with torch.no_grad():
        logits = model(adv_images)
        preds = torch.argmax(logits, dim=1)

    accepted = 0
    for i in range(N):
        if verify_with_rf(adv_images[i], int(preds[i].item())):
            accepted += 1

    results[name]["total"] = N
    results[name]["accepted"] = accepted

    rate = 100.0 * accepted / N if N > 0 else 0.0
    print(f"  Adversarial samples: {N}")
    print(f"  Accepted by rf verifier: {accepted} ({rate:.1f}%)")

print("\n=== Summary ===")
for name, r in results.items():
    N = r["total"]
    acc_rate = 100.0 * r["accepted"] / N if N > 0 else 0.0
    print(f"{name}: {r['accepted']}/{N} ({acc_rate:.1f}%) accepted")


Loading LeNet model...
Loading per-digit rf verifiers...
  Loaded rf for digit 0
  Loaded rf for digit 1
  Loaded rf for digit 2
  Loaded rf for digit 3
  Loaded rf for digit 4
  Loaded rf for digit 5
  Loaded rf for digit 6
  Loaded rf for digit 7
  Loaded rf for digit 8
  Loaded rf for digit 9
Loading test samples for attack...

Testing attacks with rf verification...

→ FGSM (L∞)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ BIM (L∞)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ BIM (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 2 (0.2%)

→ PGD (L∞)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ PGD (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ C&W (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ DeepFool (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 10 (1.0%)

=== Summary ===
FGSM (L∞): 0/1000 (0.0%) accepted
BIM (L∞): 0/1000 (0.0%) accepted
BIM 

In [5]:
import os
import torch
import numpy as np
import joblib
import foolbox as fb

# --- Setup ---
os.environ['GIT_PYTHON_GIT_EXECUTABLE'] = r'C:\Program Files\Git\bin\git.exe'
torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Load LeNet ---
print("Loading LeNet model...")
model = torch.jit.load("Models and Data splits/lenet.pt").to(device).eval()
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

# --- Load rf verifiers ---
print("Loading per-digit rf verifiers...")
rfs = {}
for digit in range(10):
    path = f"Models and Data splits/rf_digit_{digit}_pixels.joblib"
    if os.path.exists(path):
        rfs[digit] = joblib.load(path)
        print(f"  Loaded rf for digit {digit}")
    else:
        print(f"  ⚠️ No verifier for digit {digit}, will skip those predictions.")

if not rfs:
    raise RuntimeError("No rf verifiers found.")

# --- Load test samples for attack ---
print("Loading test samples for attack...")
saved_data = torch.load("Models and Data splits/selected_samples_for_attack.pt")
images = saved_data["images"].to(device)        # shape (N, 1, 28, 28), scaled [0,1]
labels = saved_data["labels"].to(device)

# --- Attack configuration ---
epsLinf = 0.2
epsL2 = 2.0
confidence_threshold = 0.90

attacks = {
    "FGSM (L∞)": fb.attacks.FGSM(),
    "BIM (L∞)": fb.attacks.LinfBasicIterativeAttack(steps=50),
    "BIM (L2)": fb.attacks.L2BasicIterativeAttack(steps=50),
    "PGD (L∞)": fb.attacks.LinfPGD(steps=100),
    "PGD (L2)": fb.attacks.L2PGD(steps=100),
    "C&W (L2)": fb.attacks.L2CarliniWagnerAttack(steps=1000),
    "DeepFool (L2)": fb.attacks.L2DeepFoolAttack(steps=500),
}

def verify_with_rf(image, predicted_digit):
    """
    Use the corresponding per-digit rf to decide if this sample is 'clean-looking'.
    Returns True if accepted, False if rejected.
    """
    if predicted_digit not in rfs:
        return False  # conservative: reject if no verifier available

    rf = rfs[predicted_digit]
    image_flat = image.detach().cpu().numpy().reshape(1, -1)
    proba_clean = rf.predict_proba(image_flat)[0, 1]  # class 1 = clean
    return proba_clean >= confidence_threshold

# --- Evaluation ---
results = {name: {"total": 0, "accepted": 0} for name in attacks}

print("\nTesting attacks with rf verification...")

for name, attack in attacks.items():
    print(f"\n→ {name}")

    eps = epsL2 if "L2" in name else epsLinf

    # Generate adversarial examples
    adv_raw, _, _ = attack(fmodel, images, labels, epsilons=eps)

    # Handle possible shape (eps, N, C, H, W)
    if adv_raw.ndim == 5:
        adv_images = adv_raw[0]
    else:
        adv_images = adv_raw

    adv_images = adv_images.to(device)
    N = adv_images.shape[0]

    with torch.no_grad():
        logits = model(adv_images)
        preds = torch.argmax(logits, dim=1)

    accepted = 0
    for i in range(N):
        if verify_with_rf(adv_images[i], int(preds[i].item())):
            accepted += 1

    results[name]["total"] = N
    results[name]["accepted"] = accepted

    rate = 100.0 * accepted / N if N > 0 else 0.0
    print(f"  Adversarial samples: {N}")
    print(f"  Accepted by rf verifier: {accepted} ({rate:.1f}%)")

print("\n=== Summary ===")
for name, r in results.items():
    N = r["total"]
    acc_rate = 100.0 * r["accepted"] / N if N > 0 else 0.0
    print(f"{name}: {r['accepted']}/{N} ({acc_rate:.1f}%) accepted")


Loading LeNet model...
Loading per-digit rf verifiers...
  Loaded rf for digit 0
  Loaded rf for digit 1
  Loaded rf for digit 2
  Loaded rf for digit 3
  Loaded rf for digit 4
  Loaded rf for digit 5
  Loaded rf for digit 6
  Loaded rf for digit 7
  Loaded rf for digit 8
  Loaded rf for digit 9
Loading test samples for attack...

Testing attacks with rf verification...

→ FGSM (L∞)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ BIM (L∞)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ BIM (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ PGD (L∞)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ PGD (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ C&W (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ DeepFool (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 10 (1.0%)

=== Summary ===
FGSM (L∞): 0/1000 (0.0%) accepted
BIM (L∞): 0/1000 (0.0%) accepted
BIM 

In [6]:
import os
import torch
import numpy as np
import joblib
import foolbox as fb

# --- Setup ---
os.environ['GIT_PYTHON_GIT_EXECUTABLE'] = r'C:\Program Files\Git\bin\git.exe'
torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Load LeNet ---
print("Loading LeNet model...")
model = torch.jit.load("Models and Data splits/lenet.pt").to(device).eval()
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

# --- Load rf verifiers ---
print("Loading per-digit rf verifiers...")
rfs = {}
for digit in range(10):
    path = f"Models and Data splits/rf_digit_{digit}_pixels.joblib"
    if os.path.exists(path):
        rfs[digit] = joblib.load(path)
        print(f"  Loaded rf for digit {digit}")
    else:
        print(f"  ⚠️ No verifier for digit {digit}, will skip those predictions.")

if not rfs:
    raise RuntimeError("No rf verifiers found.")

# --- Load test samples for attack ---
print("Loading test samples for attack...")
saved_data = torch.load("Models and Data splits/selected_samples_for_attack.pt")
images = saved_data["images"].to(device)        # shape (N, 1, 28, 28), scaled [0,1]
labels = saved_data["labels"].to(device)

# --- Attack configuration ---
epsLinf = 0.3
epsL2 = 3.0
confidence_threshold = 0.90

attacks = {
    "FGSM (L∞)": fb.attacks.FGSM(),
    "BIM (L∞)": fb.attacks.LinfBasicIterativeAttack(steps=50),
    "BIM (L2)": fb.attacks.L2BasicIterativeAttack(steps=50),
    "PGD (L∞)": fb.attacks.LinfPGD(steps=100),
    "PGD (L2)": fb.attacks.L2PGD(steps=100),
    "C&W (L2)": fb.attacks.L2CarliniWagnerAttack(steps=1000),
    "DeepFool (L2)": fb.attacks.L2DeepFoolAttack(steps=500),
}

def verify_with_rf(image, predicted_digit):
    """
    Use the corresponding per-digit rf to decide if this sample is 'clean-looking'.
    Returns True if accepted, False if rejected.
    """
    if predicted_digit not in rfs:
        return False  # conservative: reject if no verifier available

    rf = rfs[predicted_digit]
    image_flat = image.detach().cpu().numpy().reshape(1, -1)
    proba_clean = rf.predict_proba(image_flat)[0, 1]  # class 1 = clean
    return proba_clean >= confidence_threshold

# --- Evaluation ---
results = {name: {"total": 0, "accepted": 0} for name in attacks}

print("\nTesting attacks with rf verification...")

for name, attack in attacks.items():
    print(f"\n→ {name}")

    eps = epsL2 if "L2" in name else epsLinf

    # Generate adversarial examples
    adv_raw, _, _ = attack(fmodel, images, labels, epsilons=eps)

    # Handle possible shape (eps, N, C, H, W)
    if adv_raw.ndim == 5:
        adv_images = adv_raw[0]
    else:
        adv_images = adv_raw

    adv_images = adv_images.to(device)
    N = adv_images.shape[0]

    with torch.no_grad():
        logits = model(adv_images)
        preds = torch.argmax(logits, dim=1)

    accepted = 0
    for i in range(N):
        if verify_with_rf(adv_images[i], int(preds[i].item())):
            accepted += 1

    results[name]["total"] = N
    results[name]["accepted"] = accepted

    rate = 100.0 * accepted / N if N > 0 else 0.0
    print(f"  Adversarial samples: {N}")
    print(f"  Accepted by rf verifier: {accepted} ({rate:.1f}%)")

print("\n=== Summary ===")
for name, r in results.items():
    N = r["total"]
    acc_rate = 100.0 * r["accepted"] / N if N > 0 else 0.0
    print(f"{name}: {r['accepted']}/{N} ({acc_rate:.1f}%) accepted")


Loading LeNet model...
Loading per-digit rf verifiers...
  Loaded rf for digit 0
  Loaded rf for digit 1
  Loaded rf for digit 2
  Loaded rf for digit 3
  Loaded rf for digit 4
  Loaded rf for digit 5
  Loaded rf for digit 6
  Loaded rf for digit 7
  Loaded rf for digit 8
  Loaded rf for digit 9
Loading test samples for attack...

Testing attacks with rf verification...

→ FGSM (L∞)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ BIM (L∞)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ BIM (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ PGD (L∞)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ PGD (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ C&W (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 0 (0.0%)

→ DeepFool (L2)
  Adversarial samples: 1000
  Accepted by rf verifier: 10 (1.0%)

=== Summary ===
FGSM (L∞): 0/1000 (0.0%) accepted
BIM (L∞): 0/1000 (0.0%) accepted
BIM 